In [ ]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from prophet import Prophet
import warnings

C:\Users\sergi\Anaconda3\envs\cajamar\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [ ]:
warnings.simplefilter('ignore')

In [ ]:
'''
given a start date in datetime format "start_date" and an "end_date" returns a list of strings with the dates from
"start_date" to "end_date".

Example:

start_date = datetime.date(2019, 9 , 30)
end_date = datetime.date(2019, 10, 7)
get_date_range(start_date, end_date)
'''
def get_date_range(start_date, end_date):
    number_of_days = (end_date-start_date).days
    return [(start_date + datetime.timedelta(days = day)).isoformat() for day in range(number_of_days+1)]

'''
This function expects two dataframes with the same format: for the first seven columns, each column corresponds to a date 
and each row corresponds to a counter index. In position i,j there should be DELTA of counter i in date j. 
For the last two columns of the dataframes they should not reffer to a daily prediction but to the aggregated prediction 
of week_1 and week_2. Given these two dataframes (one for theprediction and one for the real values), 
the function returns de error according to the competition rules.

Examples:

import pandas as pd
import copy

test = pd.read_pickle('../data/test.pkl')

compute_error(test, test)

test_v3 = copy.copy(test)
test_v3.iloc[:,0] = test_v3.iloc[:,1]
compute_error(test_v3, test)

'''
def compute_error(pred, real):
    daily_rmses = []
    for i in range(7):
        daily_rmses.append((((real.iloc[:,i] - pred.iloc[:,i])**2/len(real.iloc[:,i])).sum())**(1/2))
    rmse_1 = sum(daily_rmses)/7
    
    first_week_pred_sum = pred.iloc[:,7].sum()
    second_week_pred_sum = pred.iloc[:,8].sum()
    first_week_real_sum = real.iloc[:,7].sum()
    second_week_real_sum = real.iloc[:,8].sum()
    
    first_week_rmse = (((first_week_real_sum - first_week_pred_sum)**2)/len(real.iloc[:,7]))**(1/2)
    second_week_rmse = (((second_week_real_sum - second_week_pred_sum)**2)/len(real.iloc[:,8]))**(1/2)
    rmse_2 = (first_week_rmse + second_week_rmse)/2
    
    return (rmse_1 + rmse_2)/2

In [ ]:
path = '../data/df6.pkl'

df = pd.read_pickle(path)
start_date = datetime.date(2019, 2 , 1)
end_date = datetime.date(2020, 1, 17)
train = df[df['DATE'].isin(get_date_range(start_date, end_date))]
train = train[train['IS_GOOD']==1]
train = train[['ID','DATE','DELTA']]

start_date = datetime.date(2020, 1 , 18)
end_date = datetime.date(2020, 1, 31)
test = df[df['DATE'].isin(get_date_range(start_date, end_date))]
test = test[test['IS_GOOD']==1]
test = test[['ID','DATE','DELTA']]

print('Train:', train.shape, 'Test:', test.shape)

In [ ]:
results_dict = {}
for i in tqdm(train['ID'].unique()):
    one_counter_train = train[train['ID']==i]
    one_counter_train.drop('ID', axis=1, inplace=True)
    one_counter_train.rename(columns = {'DATE':'ds', 'DELTA':'y'}, inplace = True)
    model = Prophet(yearly_seasonality=False, daily_seasonality=False)
    model.fit(one_counter_train)
    future = model.make_future_dataframe(periods=14)
    forecast = model.predict(future)
    results_dict[i] = forecast['yhat'].tail(14)

In [50]:
results_df = pd.DataFrame.from_dict(results_dict)
results_df = results_df.T
results_df.columns = [['Dia_1','Dia_2','Dia_3','Dia_4','Dia_5','Dia_6','Dia_7',
                       'Dia_8','Dia_9','Dia_10','Dia_11','Dia_12','Dia_13','Dia_14']]
results_df['Semana_1'] = results_df[['Dia_1','Dia_2','Dia_3','Dia_4','Dia_5','Dia_6','Dia_7']].sum(axis=1)
results_df['Semana_2'] = results_df[['Dia_8','Dia_9','Dia_10','Dia_11','Dia_12','Dia_13','Dia_14']].sum(axis=1)
results_df.drop(['Dia_8','Dia_9','Dia_10','Dia_11','Dia_12','Dia_13','Dia_14'], axis=1, inplace=True)

In [51]:
test = pd.read_pickle('../data/test.pkl')
error = compute_error(results_df, test)
print('Prophet:', round(error,2))

Prophet: 43987.68
